In [1]:
import nibabel as nib
import os
import glob
import pandas as pd
import nilearn.plotting as nilp
from matplotlib import pyplot as plt
import numpy as np
from boutiques import descriptor2func as fun

pd.set_option("display.max_rows", None)

In [2]:
# Read QC file from experiments 1 and 2
qc_df = pd.read_csv(
    "status-4554467823934740347_nlr4_dbm8_from_nifti_old_pipeline_qc.csv"
)
subj_ids = qc_df["subject"].to_numpy()

# Check that we get the same number of subject as in the paper
assert len(subj_ids) == 344

# Filter subjects that passed QC for exp 1 and 2
subj_ids_pass = qc_df[qc_df["pipeline_complete"] == "SUCCESS"]["subject"].to_numpy()
assert len(subj_ids_pass) == 306

In [3]:
# The mnc files in mincignore.csv need to be ignored as they were not involved in the experiment
# It is because some subjects had more than one T1 in the same session.
# When that happened Michelle looked at the T1s on LONI and chose one to keep. The paths to the T1s that were not used are in mincignore.csv
with open(os.path.join("input", "mincignore.csv")) as f:
    ignored_files = f.readlines()
ignored_files = [
    x.replace("/data/origami/livingpark/zeighami-etal-2019/dbm/", "").replace("\n", "")
    for x in ignored_files
]

In [4]:
def get_mincs(subj_id):
    """
    Return (minc_file, minc_from_nifti_file) for subject id. Minc_file
    is the path to the MINC file converted from DICOM whereas minc_from_nifti_file
    is the path to the MINC file converted from Nifti.
    """

    def file_in_dir(dirname):
        if not os.path.exists(dirname):
            print(f"Warning: missing directory: {dirname}")
            return None
        g = [x for x in glob.glob(os.path.join(dirname, "*")) if x not in ignored_files]
        if len(g) == 0:
            raise Exception(f"Empty directory: {dirname}")
        if len(g) != 1:
            raise Exception(f"Found directory with more than 1 file: {dirname}")
        return g[0]

    # For this subject there's multiple minc files converted from nifti
    # Michelle said Nikhil said that the DICOM to Nifti conversion might have been
    # bad and that we should use the *_T1w1* file
    if subj_id == 3869:
        return (
            "input/minc/3869/1/557573.mnc",
            "input/minc_from_nifti/3869/1/sub-3869_ses-1_acq-grappa2_run-05_T1w1.mnc",
        )

    minc_file = file_in_dir(os.path.join("input", "minc", str(subj_id), "1"))
    minc_from_nifti_file = file_in_dir(
        os.path.join("input", "minc_from_nifti", str(subj_id), "1")
    )

    return minc_file, minc_from_nifti_file

In [5]:
def get_shape(minc_file):
    """
    Get the shape of the image in minc_file
    """
    if minc_file is None:
        return None
    try:
        im = nib.load(minc_file)
    except ValueError as e:
        print(f"Warning: cannot read MINC file {minc_file}: {e}")
        return None
    return im.header.get_data_shape()


def get_type(minc_file):
    """
    Get the data type of the image in minc_file
    """
    if minc_file is None:
        return None
    try:
        im = nib.load(minc_file)
    except ValueError as e:
        print(f"Warning: cannot read MINC file {minc_file}: {e}")
        return None
    return im.header.get_data_dtype()

In [6]:
# Create a dataframe with the following columns
# PATNO: patient identifier
# minc_files: (minc_file, minc_file_from_nifti) as returned by function get_mincs
# shape1: shape of minc_file
# shape2: shape of minc_file_from_nifti
# dtype1: data type of minc_file
# dtype2: data type of minc_file_from_nifti
df = pd.DataFrame(
    {
        "PATNO": subj_ids_pass,
        "minc_files": [get_mincs(s) for s in subj_ids_pass],
    }
)
df["shape1"] = df["minc_files"].apply(lambda x: get_shape(x[0]))
df["shape2"] = df["minc_files"].apply(lambda x: get_shape(x[1]))

df["dtype1"] = df["minc_files"].apply(lambda x: get_type(x[0]))
df["dtype2"] = df["minc_files"].apply(lambda x: get_type(x[1]))

In [7]:
# Ignore images for which a MINC file couldn't be read by nibabel
df = df[~df["shape1"].isna() & ~df["shape2"].isna()]

In [8]:
# A few images had unusually low dimensions (x or y or z < 100) compared to the other ones
# Here we verified that these images were filterd by QC in experiment 5 and 6 and 
# were therefore excluded from the study


def filter_fn(row, file_id):
    s = row[f"shape{file_id}"]
    for i in (0, 1, 2):
        if s[i] <= 100:
            return True
    return False


def filter_fn_minc(row):
    return filter_fn(row, 1)


def filter_fn_minc_from_nifti(row):
    return filter_fn(row, 2)


m = df.apply(filter_fn_minc, axis=1)
corrupt_mincs = df[m]["PATNO"].to_numpy()
print(
    f"Found {len(corrupt_mincs)} corrupted MINC files. Note: these files were included in experiments 1 and 2."
)

# verify that corrupt mincs failed qc in exp 5 and 6
qc_exp5_6 = pd.read_csv("status-_460520092359662532_old_pipeline_qc.csv")
for x in corrupt_mincs:
    assert (
        x in qc_exp5_6[qc_exp5_6["pipeline_complete"] == "FAIL"]["subject"].to_numpy()
    ), x

m = df.apply(filter_fn_minc_from_nifti, axis=1)
assert len(df[m]) == 0
print(
    "All images with unusually low dimension were filtered by QC in experiments 5 and 6"
)

Found 2 corrupted MINC files. Note: these files were included in experiments 1 and 2.
All images with unusually low dimension were filtered by QC in experiments 5 and 6


In [9]:
# Remove images with unusually low dimension
m = df.apply(filter_fn_minc, axis=1)
df = df[~m]

In [10]:
def check_reoriented(x):
    """
    x = (minc_file, minc_file_from_nifti)

    This function reorients the MINC file in the second element of x (minc_file_from_nifti) to (hopefully) give it
    the same orientation as the file in the first element of x (minc_file).

    The function also checks that the resulting file is nearly identical to minc_file.
    """
    f1, f2 = x
    print(".", end="")
    i1 = nib.load(f1)
    i2 = nib.load(f2)

    #     # Reorient i2 as i1
    #     Nifti version, doesn't work because nibabel can't write to mnc so
    #     we have to use nii2mnc that changes the orientation back to what it was
    #     # See https://github.com/nipy/nibabel/issues/1010
    #     from nibabel.orientations import ornt_transform
    #     orig_ornt = nib.io_orientation(i2.affine)
    #     targ_ornt = nib.io_orientation(i1.affine)

    #     transform = ornt_transform(orig_ornt, targ_ornt)
    #     i2_orient = i2.as_reoriented(transform)

    # Call mincreshape on f2
    mincreshape = fun.function("10.5281/zenodo.7988259")
    filename = f2.replace("minc_from_nifti", "minc_from_nifti_reorient_minc")
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    out = mincreshape(
        "launch",
        infile=f2,
        outfile_name=filename,
        dimsize="zspace=-1",
        flip_ydirection=True,
        flip_zdirection=True,
        dimorder="xspace,zspace,yspace",
    )
    if out.exit_code != 0:
        print(f"Error running mincreshape: {out.stderr}")
        return None

    # Load the reshaped file and checked that it's nearly identical to minc_file
    i2_orient = nib.load(filename)
    sanity_check = np.all(np.abs(i1.get_fdata() - i2_orient.get_fdata()) < 0.5)

    if sanity_check:
        return filename

    print(f"Sanity check failed for {filename}, trying to flip xdirection too.")

    # Re-run mincreshape, this time with -xdirection
    os.remove(filename)
    out = mincreshape(
        "launch",
        infile=f2,
        outfile_name=filename,
        dimsize="zspace=-1",
        flip_xdirection=True,
        flip_ydirection=True,
        flip_zdirection=True,
        dimorder="xspace,zspace,yspace",
    )
    if out.exit_code != 0:
        print(f"Error running mincreshape: {out.stderr}")
        return None

    # Load the reshaped file and checked that it's nearly identical to minc_file
    i2_orient = nib.load(filename)
    sanity_check = np.all(np.abs(i1.get_fdata() - i2_orient.get_fdata()) < 0.5)

    if sanity_check:
        return filename

    print(f"Sanity check failed again for {filename}, giving up.")
    return None

In [11]:
df["reoriented"] = df["minc_files"].apply(check_reoriented)

.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3102/1/sub-3102_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487

.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3171/1/sub-3171_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3172/1/sub-3172_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3173/1/sub-3173_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3174/1/sub-3174_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3175/1/sub-3175_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23

.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3354/1/sub-3354_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3355/1/sub-3355_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3357/1/sub-3357_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3358/1/sub-3358_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3360/1/sub-3360_ses-1_run-01_T1w.mnc, trying to 

.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3752/1/sub-3752_ses-1_run-01

.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3814/1/sub-3814_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3815/1/sub-3815_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3816/1/sub-3816_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3817/1/sub-3817_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/3818/1/sub-3818_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23

Sanity check failed for input/minc_from_nifti_reorient_minc/4024/1/sub-4024_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/4025/1/sub-4025_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/4026/1/sub-4026_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/4027/1/sub-4027_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23.0.5, build bc4487a
Sanity check failed for input/minc_from_nifti_reorient_minc/4029/1/sub-4029_ses-1_run-01_T1w.mnc, trying to flip xdirection too.
Docker version 23.0.5, build bc4487a
.Docker version 23